In [1]:
import numpy as np
import pandas as pd
import cv2
import os

In [2]:
number_epoch=50
step_per_epoch_training=20
step_per_epoch_validation=20

batch_size_training=15
batch_size_testing=2
batch_size_validation=2

In [3]:
import keras
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.models import Sequential
from keras import optimizers
from keras import regularizers
from keras.layers.normalization import BatchNormalization 

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [13]:
def cnn(input_shape=(200,150,3)):
    model=Sequential()
    model.add(Conv2D(256,kernel_size=(4,4),activation="relu",input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,4)))
    model.add(Conv2D(128,(3,5),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64,(2,2),activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Dropout(.5))
    model.add(Dense(64))
    model.add(Dense(2,activation="softmax"))
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    model.summary()
    return model

In [14]:
model=cnn()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 197, 147, 256)     12544     
_________________________________________________________________
batch_normalization_5 (Batch (None, 197, 147, 256)     1024      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 98, 36, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 96, 32, 128)       491648    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 16, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 48, 16, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 47, 15, 64)        32832     
__________

In [15]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [16]:
data_generator=ImageDataGenerator(preprocessing_function=preprocess_input)

In [17]:
train_generator=data_generator.flow_from_directory("data/train",
                                                   target_size=(200,150),
                                                   batch_size=batch_size_training,
                                                   class_mode="categorical")

Found 2637 images belonging to 2 classes.


In [18]:
validation_generator=data_generator.flow_from_directory("data/test",
                                                   target_size=(200,150),
                                                   batch_size=batch_size_validation,
                                                   class_mode="categorical")

Found 660 images belonging to 2 classes.


In [20]:
history=model.fit_generator(train_generator,
                            steps_per_epoch=step_per_epoch_training,
                            epochs=number_epoch,
                            validation_data=validation_generator,
                            validation_steps=step_per_epoch_validation )

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
20/20 [==============================] - 114s 6s/step - loss: 4.1803 - acc: 0.6467 - val_loss: 5.3512 - val_acc: 0.6500
Epoch 2/50
20/20 [==============================] - 108s 5s/step - loss: 4.4338 - acc: 0.7000 - val_loss: 5.1383 - val_acc: 0.6750
Epoch 3/50
20/20 [==============================] - 105s 5s/step - loss: 4.0201 - acc: 0.7233 - val_loss: 2.0318 - val_acc: 0.7500
Epoch 4/50
20/20 [==============================] - 106s 5s/step - loss: 3.7679 - acc: 0.7433 - val_loss: 5.1211 - val_acc: 0.6750
Epoch 5/50
20/20 [==============================] - 106s 5s/step - loss: 5.1011 - acc: 0.6633 - val_loss: 6.8599 - val_acc: 0.5500
Epoch 6/50
20/20 [==============================] - 107s 5s/step - loss: 4.4527 - acc: 0.7067 - val_loss: 5.2384 - val_acc: 0.6750
Epoch 7/50
20/20 [==============================] - 105s 5s/step - loss: 4.1325 - acc: 0.7367 - val_loss: 5.2384 - val_acc: 0.6750
Epoch 8/50
20/20 [=================

KeyboardInterrupt: 

In [ ]:
model.save("model.hdf5")